In [ ]:
import pandas as pd
import os
from google.cloud import bigquery
import cmapBQ.query as cmap_query
import cmapBQ.config as cmap_config
from cmapPy.pandasGEXpress.write_gctx import write as write_gctx

#viewing
#import qgrid


# Set up credentials
bq_client = cmap_config.get_bq_client()

#Old method of authentication
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'cmap-big-table-bd0276aaff22.json'
#bq_client = bigquery.Client()

In [ ]:
moas = cmap_query.list_cmap_moas(bq_client)
display(moas)

In [ ]:
brom_inhib = cmap_query.cmap_compounds(client=bq_client, moa='Bromodomain inhibitor')

In [ ]:
display(brom_inhib)

<div style="font-size: 12pt;line-height:30px">

In another example, we can query the dataset to look for all available gene targets using list_cmap_targets(). From this list of targets, we will see if our desired target **'CDK1'** is witin the database. 

In [ ]:
targets = cmap_query.list_cmap_targets(bq_client)
display(targets)

In [ ]:
'CDK1' in targets.target.unique()

<div style="font-size: 12pt;line-height:30px">

If the desired target is in the database, we can then query the compound table to get information about what compounds affect the CDK1 target. 

In [ ]:
CDK1_cpinfo = cmap_query.cmap_compounds(client=bq_client, target='CDK1', verbose=True)
display(CDK1_cpinfo)

<div style="font-size: 12pt;line-height:30px">

Lets take the first 10 compounds and see how many signatures are available for those compounds. We can pass a list of compounds to the **cmap_sig function**, which then queries the dataset for compounds that match.

In [ ]:
CDK1_cps = CDK1_cpinfo.pert_id.unique()

In [ ]:
CDK1_cps[1:10]

In [ ]:
CDK1_siginfo = cmap_query.cmap_sig(bq_client, pert_id=list(CDK1_cps), verbose=True)

In [ ]:
CDK1_siginfo

<div style="font-size: 12pt;line-height:30px">


The siginfo file provides information on the conditions for each experiment such as compound, dose, timepoint, cell line, and more.

The table also includes information regarding the signal strength and replicate correlation of the compound. The `distil_tas` contains the signatures **Transcriptional Activity Score (TAS)** which is an aggregate measure of strength and reproducibilty.  [More information about signature quality metrics can be found on Connectopedia](https://clue.io/connectopedia/signature_quality_metrics)

In [ ]:
print("Time points:\n {}".format(CDK1_siginfo.pert_itime.unique()))
print("Doses: \n {}".format(CDK1_siginfo.pert_idose.unique()))

In [ ]:
filtered_CDK1 = CDK1_siginfo.loc[
    (CDK1_siginfo.pert_dose == 10 ) & 
    (CDK1_siginfo.pert_itime == '24 h' )
]
display(filtered_CDK1[0:10])

<div style="font-size: 12pt;line-height:30px">

From this table if we want the numerical data, we can extract the sig_ids and use them to query Level 5 database

In [ ]:
CDK1_sig_ids = list(filtered_CDK1.sig_id.unique())

In [ ]:
CDK1_data = cmap_query.cmap_matrix(bq_client, data_level='level5', cid=list(CDK1_siginfo.sig_id.unique()[0:1000]))

In [ ]:
CDK1_data.data_df

In [ ]:
#write_gctx(CDK1_data, filename)